In [ ]:
import torch
import torch.onnx
import sys
import onnx
import onnxruntime
import numpy as np

from ros_packages.px4_ros_extended.src_py.DDPG.models import Actor_small_sep_head  # Change model

 # https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html

In [ ]:
state_dim = 6  # Change state dimension
action_dim = 3  # Change action dimension
actor = Actor_small_sep_head(state_dim, action_dim).float()  # Change model

In [ ]:
actor.load_state_dict(torch.load('<model_folder>/<filename>_actor.pt'))

In [ ]:
actor.eval()

In [ ]:
batch_size = 1
x = torch.randn(batch_size, state_dim, requires_grad=True)
torch_out = actor(x)

In [ ]:
# Export the model
torch.onnx.export(actor,                       # model being run
                  x,                           # model input (or a tuple for multiple inputs)
                  "test_onnx_model.onnx",  # where to save the model (can be a file or file-like object)
                  export_params=True,          # store the trained parameter weights inside the model file
                  opset_version=10,            # the ONNX version to export the model to
                  do_constant_folding=True,    # whether to execute constant folding for optimization
                  input_names = ['input'],     # the model's input names
                  output_names = ['output'])   # the model's output names

In [ ]:
onnx_model = onnx.load("test_onnx_model.onnx")
onnx.checker.check_model(onnx_model)

In [ ]:
ort_session = onnxruntime.InferenceSession("test_onnx_model.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Ok!")

In [ ]:
x = torch.randn(batch_size, state_dim, requires_grad=True)
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)
pred = ort_outs[0]

In [ ]:
pred